In [1]:
!pip install mediapipe
!pip install opencv-python


DEPRECATION: Loading egg at c:\users\franf\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\users\franf\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
import cv2
import mediapipe as mp

# Inicializar BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen
image = cv2.imread('./yolo/videoframe_0.jpg.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Realizar la detección de pose
results = pose.process(image_rgb)

# Dibujar las anotaciones de la pose en la imagen
if results.pose_landmarks:
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

# Mostrar la imagen con las poses detectadas
cv2.imshow('BlazePose Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Liberar recursos
pose.close()


AttributeError: 'NoneType' object has no attribute 'shape'